In [1]:
pip install --upgrade kagglehub kagglesdk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import numpy as np
import xgboost as xgb
import pickle
import cv2
import os
from pathlib import Path

# 1. SETUP & LOAD ASSETS
print("Loading Model Components...")

# Load the Class Names
with open('class_labels.pkl', 'rb') as f:
    class_names = pickle.load(f)

# Load the CNN (Load without compiling to avoid optimizer warnings)
full_model = tf.keras.models.load_model("Final_OCT_EfficientNet.keras", compile=False)

# Create the Feature Extractor
feature_extractor = tf.keras.models.Model(
    inputs=full_model.input, 
    outputs=full_model.get_layer("feature_extraction_layer").output
)

# Load the XGBoost Classifier
xgb_model = xgb.XGBClassifier()
xgb_model.load_model("Final_XGBoost_Classifier.json")

# 2. FIND DATA PATH
print("Locating Test Images...")

# Define the base path for kagglehub downloads on Windows
base_path = Path("C:/Users/adim/.cache/kagglehub/datasets/anirudhcv/labeled-optical-coherence-tomography-oct/versions")

if base_path.exists():
    # Identify the most recent version folder
    latest_version = sorted([d for d in base_path.iterdir() if d.is_dir()])[-1]
    test_dir = latest_version / "Dataset - train+val+test" / "test"
    print(f"Dataset located at: {test_dir}")
else:
    # Fallback to current working directory
    test_dir = Path("./Dataset - train+val+test/test")
    print(f"Kagglehub path not found, trying local path: {test_dir}")

# Select a sample image from the DRUSEN folder
sample_images = list((test_dir / "DRUSEN").glob("*.jpeg"))

if not sample_images:
    print("Error: No images found. Check your dataset path.")
else:
    img_path = sample_images[0]
    print(f"Processing image: {img_path.name}")

    # 3. IMAGE PREPROCESSING
    img = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (224, 224))
    
    x = tf.cast(img_resized, tf.float32)
    x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
    x = np.expand_dims(x, axis=0) 

    # 4. HYBRID PREDICTION PIPELINE
    print("Extracting Deep Features...")
    deep_features = feature_extractor.predict(x, verbose=0)
    
    print("Generating Decision...")
    preds_proba = xgb_model.predict_proba(deep_features)
    
    result_index = np.argmax(preds_proba)
    confidence = np.max(preds_proba)

    # 5. OUTPUT RESULTS
    print("\n" + "="*30)
    print("   AI DIAGNOSTIC REPORT")
    print("="*30)
    print(f"Detected Condition: {class_names[result_index]}")
    print(f"Confidence Level:   {confidence:.2%}")
    print("="*30)

Loading Model Components...
Locating Test Images...
Dataset located at: C:\Users\adim\.cache\kagglehub\datasets\anirudhcv\labeled-optical-coherence-tomography-oct\versions\2\Dataset - train+val+test\test
Processing image: DRUSEN-1001666-7.jpeg
Extracting Deep Features...
Generating Decision...

   AI DIAGNOSTIC REPORT
Detected Condition: NORMAL
Confidence Level:   74.85%
